# Optimizing ByteTrack parameters

In [10]:
import cv2
import numpy as np
from ouster import client
from ouster import pcap
from ultralytics import YOLO
from ouster.client import Scans, XYZLut, SensorInfo, destagger
from contextlib import closing
from collections import defaultdict

model = YOLO('../weights/best_3000_s.pt')

metadata_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.json"
pcap_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.pcap"

with open(metadata_path, 'r') as f:
    metadata = client.SensorInfo(f.read())

fps = int(str(metadata.mode)[-2:])
width = int(str(metadata.mode)[:4])
height = int(str(metadata.prod_line)[5:])

pcap_file = pcap.Pcap(pcap_path, metadata)

xyz_lut = client.XYZLut(metadata)

track_history = defaultdict(lambda: [])

with closing(Scans(pcap_file)) as scans:

    save_path = "C:/Users/szyme/Ouster/Dangerous-situations-with-pedastrians/results_mp4"
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    i = 0
    for scan in scans:
        i += 1
        sig_field = scan.field(client.ChanField.SIGNAL)
        sig_destaggered = destagger(metadata, sig_field)
        scaling_factor = 0.004
        scaled_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered)
        signal_image = scaled_arr.astype(np.uint8)
        combined_img = np.dstack((signal_image, signal_image, signal_image))

        xyz_destaggered = client.destagger(metadata, xyz_lut(scan))

        results = model.track(source=combined_img, persist=True, imgsz=1024, tracker='bytetrack.yaml')

        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        speed = results[0].speed
        total_speed = 
        print(speed)

        if (results[0].boxes.id == None):
            ids = ''
        else:
            ids = results[0].boxes.id.cpu().numpy().astype(int)

        # iteration through identified objects
        for box, id in zip(boxes, ids):
            center_x, center_y = int((box[0] + box[2])/2), int((box[1] + box[3])/2) #
            xyz_val = xyz_destaggered[(center_y, center_x)]
            track = track_history[id]
            track.append(xyz_val)

        annotated_frame = results[0].plot()

        if True:
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
            cv2.waitKey(1)  # 1 millisecond

        vid_writer.write(annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        if i >= 2000:
            break

    vid_writer.release()
    cv2.destroyAllWindows()


0: 128x1024 (no detections), 59.6ms
Speed: 0.0ms preprocess, 59.6ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 55.2ms
Speed: 0.0ms preprocess, 55.2ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 1024)


{'preprocess': 0.0, 'inference': 59.63134765625, 'postprocess': 0.0}
{'preprocess': 0.0, 'inference': 55.193424224853516, 'postprocess': 0.0}



0: 128x1024 2 humanss, 57.2ms
Speed: 0.0ms preprocess, 57.2ms inference, 1.2ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 58.0ms
Speed: 1.0ms preprocess, 58.0ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 59.9ms


{'preprocess': 0.0, 'inference': 57.18994140625, 'postprocess': 1.209259033203125}
{'preprocess': 1.0251998901367188, 'inference': 58.00962448120117, 'postprocess': 1.0018348693847656}


Speed: 1.0ms preprocess, 59.9ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 59.1ms
Speed: 1.0ms preprocess, 59.1ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 57.4ms


...

{'preprocess': 0.0, 'inference': 54.334402084350586, 'postprocess': 0.0}
{'preprocess': 1.0025501251220703, 'inference': 54.587364196777344, 'postprocess': 1.0023117065429688}
{'preprocess': 0.0, 'inference': 52.492618560791016, 'postprocess': 0.0}


0: 128x1024 (no detections), 60.1ms
Speed: 0.0ms preprocess, 60.1ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 (no detections), 57.9ms
Speed: 1.4ms preprocess, 57.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 (no detections), 52.3ms
Speed: 1.0ms preprocess, 52.3ms inference, 1.1ms postprocess per image at shape (1, 3, 128, 1024)



{'preprocess': 0.0, 'inference': 60.147762298583984, 'postprocess': 1.0023117065429688}
{'preprocess': 1.4159679412841797, 'inference': 57.93046951293945, 'postprocess': 0.0}
{'preprocess': 1.0027885437011719, 'inference': 52.300453186035156, 'postprocess': 1.1267662048339844}


0: 128x1024 (no detections), 56.1ms
Speed: 1.5ms preprocess, 56.1ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 (no detections), 54.2ms
Speed: 1.0ms preprocess, 54.2ms inference, 1.4ms postprocess per image at shape (1, 3, 128, 1024)


{'preprocess': 1.5079975128173828, 'inference': 56.14447593688965, 'postprocess': 0.0}
{'preprocess': 1.0030269622802734, 'inference': 54.19611930847168, 'postprocess': 1.3697147369384766}
